In [ ]:
import panel as pn
import param

# CSS pour le Stepper
css = """
.stepper {
    display: flex;
    justify-content: space-between;
    margin-bottom: 20px;
}
.step {
    text-align: center;
    position: relative;
}
.step .circle {
    width: 40px;
    height: 40px;
    border-radius: 50%;
    background-color: #d3d3d3;
    display: flex;
    justify-content: center;
    align-items: center;
    margin: 0 auto;
    color: white;
    font-weight: bold;
}
.step.completed .circle {
    background-color: #4CAF50;
}
.step.completed .circle:after {
    content: '✓';
    font-size: 24px;
}
.step .label {
    margin-top: 10px;
}
.step:not(:last-child)::after {
    content: '';
    position: absolute;
    width: calc(100% - 40px);
    height: 2px;
    background-color: #d3d3d3;
    top: 20px;
    left: 50%;
    z-index: -1;
}
.step.completed:not(:last-child)::after {
    background-color: #4CAF50;
}
"""

# Injecter le CSS dans l'application Panel
pn.extension(raw_css=[css])

class Stepper(param.Parameterized):
    current_step = param.Integer(default=0, bounds=(0, 3))

    def __init__(self, **params):
        super().__init__(**params)
        self.step_names = ["Shopping basket", "Personal details", "Shipping details", "Confirmation"]
        self.steps = [self.step_view(step) for step in self.step_names]
        self.next_button = pn.widgets.Button(name="Next", button_type="primary")
        self.prev_button = pn.widgets.Button(name="Previous", button_type="default")
        self.next_button.on_click(self.next_step)
        self.prev_button.on_click(self.prev_step)
        self.view = pn.Column(
            self.stepper_view,
            self.steps[self.current_step],
            pn.Row(self.prev_button, self.next_button),
            sizing_mode="stretch_width"
        )
        self.update_buttons()

    @param.depends("current_step")
    def stepper_view(self):
        steps_html = ""
        for i, step_name in enumerate(self.step_names):
            completed_class = "completed" if i <= self.current_step else ""
            steps_html += f'''
                <div class="step {completed_class}">
                    <div class="circle">{i+1}</div>
                    <div class="label">{step_name}</div>
                </div>
            '''
        return pn.pane.HTML(f'<div class="stepper">{steps_html}</div>')

    def step_view(self, step_name):
        return pn.pane.Markdown(f"### {step_name}")

    def next_step(self, event):
        if self.current_step < len(self.steps) - 1:
            self.current_step += 1
        self.update_buttons()

    def prev_step(self, event):
        if self.current_step > 0:
            self.current_step -= 1
        self.update_buttons()

    def update_buttons(self):
        self.next_button.disabled = self.current_step == len(self.steps) - 1
        self.prev_button.disabled = self.current_step == 0

stepper = Stepper()
app = pn.Column(
    pn.pane.Markdown("## Stepper Example"),
    stepper.view
)

app.servable()
app.show()

In [7]:
#save app to html standalone file
app.save('stepper.html')